In [18]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from tqdm import tqdm
import sqlite3
import pickle
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Get data from database and import it into a csv file

database_path = "./mydatabase.sqlite"

# connect to database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Fetch data from table
query_maotai = "SELECT * FROM MAOTAI"
maotai = pd.read_sql_query(query_maotai, conn)

query_hs300 = "SELECT * FROM HS300"
hs300 = pd.read_sql_query(query_hs300, conn)

query_bond10 = "SELECT * FROM BOND10"
bond10 = pd.read_sql_query(query_bond10, conn)

query_credit = "SELECT * FROM CREDIT"
credit = pd.read_sql_query(query_credit, conn)

query_zz500 = "SELECT * FROM ZZ500"
zz500 = pd.read_sql_query(query_zz500, conn)

query_bond3_5 = "SELECT * FROM 'BOND3-5'"
bond3_5 = pd.read_sql_query(query_bond3_5, conn)

# close the connection with database
cursor.close()
conn.close()

# Delete the line containing missing values
maotai.dropna(inplace=True)
hs300.dropna(inplace=True)
bond3_5.dropna(inplace=True)
bond10.dropna(inplace=True)
zz500.dropna(inplace=True)
credit.dropna(inplace=True)

# Rename the column of DataFrame
maotai.rename(columns={"mt_close": "Close"}, inplace=True)
hs300.rename(columns={"hs300": "Close"}, inplace=True)
bond10.rename(columns={"bond10": "Close"}, inplace=True)
bond3_5.rename(columns={"bond3-5": "Close"}, inplace=True)
credit.rename(columns={"credit": "Close"}, inplace=True)
zz500.rename(columns={"zz500": "Close"}, inplace=True)

# Replace missing values with Nan, as well as convert STRING type to FLOAT type
maotai["Close"] = maotai["Close"].replace('', float('nan'))
maotai["Close"] = maotai["Close"].astype(float)

hs300["Close"] = hs300["Close"].replace('', float('nan'))
hs300["Close"] = hs300["Close"].astype(float)

bond10["Close"] = bond10["Close"].replace('', float('nan'))
bond10["Close"] = bond10["Close"].astype(float)

bond3_5["Close"] = bond3_5["Close"].replace('', float('nan'))
bond3_5["Close"] = bond3_5["Close"].astype(float)

credit["Close"] = credit["Close"].replace('', float('nan'))
credit["Close"] = credit["Close"].astype(float)

zz500["Close"] = zz500["Close"].replace('', float('nan'))
zz500["Close"] = zz500["Close"].astype(float)

rm = hs300["Close"]
rf = 0.02

maotai["Date"] = pd.to_datetime(maotai["Date"])

hs300['Date'] = pd.to_datetime(hs300['Date'])

zz500['Date'] = pd.to_datetime(zz500['Date'])

bond10['Date'] = pd.to_datetime(bond10['Date'])

bond3_5['Date'] = pd.to_datetime(bond3_5['Date'])

credit['Date'] = pd.to_datetime(credit['Date'])

# maotai.plot(x="Date", y="Close", title="Maotai Stock Close Price")
# hs300.plot(x="Date", y="Close", title="HS300 Index Close Price")
# # plt.show()

maotai["Close"].fillna(0, inplace=True)
credit["Close"].fillna(0, inplace=True)
bond3_5["Close"].fillna(0, inplace=True)
bond10["Close"].fillna(0, inplace=True)
zz500["Close"].fillna(0, inplace=True)
hs300["Close"].fillna(0, inplace=True)

maotai["Close"].interpolate(method='linear', inplace=True)

std = maotai["Close"].std()
mean = maotai["Close"].mean()
maotai["Close"] = np.where(maotai["Close"] > mean + 3 * std, mean + 3 * std, maotai["Close"])
maotai["Close"] = np.where(maotai["Close"] < mean - 3 * std, mean - 3 * std, maotai["Close"])

maotai["z-score"] = maotai["Close"].rolling(window=60).apply(lambda x: (x[-1] - x.mean()) / x.std(), raw=True)

maotai.set_index("Date", inplace=True)
maotai_monthly = maotai.resample("M").last()
maotai_monthly["Momentum"] = (
        maotai_monthly["Close"].pct_change(periods=12) - maotai_monthly["Close"].pct_change(periods=1)).dropna()

maotai_yearly = maotai.resample("Y").last()
maotai_yearly["Yearly Return"] = maotai_yearly["Close"].pct_change()

hs300.set_index('Date', inplace=True)
zz500.set_index('Date', inplace=True)
bond10.set_index('Date', inplace=True)
bond3_5.set_index('Date', inplace=True)
credit.set_index('Date', inplace=True)

maotai.columns = maotai.columns.astype(str)
hs300.columns = hs300.columns.astype(str)
zz500.columns = zz500.columns.astype(str)
bond10.columns = bond10.columns.astype(str)
bond3_5.columns = bond3_5.columns.astype(str)
credit.columns = credit.columns.astype(str)

maotai_monthly = maotai.resample("M").last()
hs300_monthly = hs300.resample("M").last()
zz500_monthly = zz500.resample("M").last()
bond10_monthly = bond10.resample("M").last()
bond3_5_monthly = bond3_5.resample("M").last()
credit_monthly = credit.resample("M").last()

# result = pd.concat([[hs300_monthly][zz500_monthly]], axis=0, ignore_index=True)
# result = np.concatenate((maotai_monthly, hs300_monthly, zz500_monthly, bond10_monthly, bond3_5_monthly, credit_monthly), axis=1)
# result = pd.merge(pd.merge(maotai, hs300, on='Date', how='inner'), zz500, on='Date', how='inner')
# result = pd.merge(result, bond10, on='Date', how='inner')
result = pd.concat([hs300, bond10, bond3_5, credit, zz500], axis=1,
                   keys=['hs300', 'bong10', 'bond3_5', 'credit', 'zz500'])
result.fillna(0, inplace=True)

# Analyze 5 stocks(Closing prices from 2015-01-01 to 2021-12-31) and calculate the logarithmic rate of return
# Code, a string, is a name of the stock.
def get_ret(code, data):
    # data = ak.stock_zh_a_hist(symbol=code, period="daily", start_date="20150101", end_date='20211231', adjust="")
    # data.index = pd.to_datetime(data['Date'], format='%Y-%m-%d')  # Set the date index
    close = data[code]  # Daily closing price
    close.name = code
    ret = np.log(close / close.shift(1))  # Daily Return Rate
    return ret


codes = ['hs300', 'bong10', 'bond3_5', 'credit', 'zz500']
ret = pd.DataFrame()
for code in codes:
    ret_ = get_ret(code, result)
    ret = pd.concat([ret, ret_], axis=1)
ret.replace([np.inf, -np.inf], np.nan, inplace=True)
ret = ret.dropna()

R_cov = ret.cov()  # covariance
cov = np.array(R_cov)


def risk_budget_objective(weights, cov):
    weights = np.array(weights) 
    sigma = np.sqrt(np.dot(weights, np.dot(cov, weights)))  # Gets the combined standard deviation
    # sigma = np.sqrt(weights@cov@weights)
    MRC = np.dot(cov, weights) / sigma  # MRC = cov@weights/sigma
    # MRC = np.dot(weights,cov)/sigma
    TRC = weights * MRC
    delta_TRC = [sum((i - TRC) ** 2) for i in TRC]
    return sum(delta_TRC)


def total_weight_constraint(x):
    return np.sum(x) - 1.0


x0 = np.ones(cov.shape[0]) / cov.shape[0]
bnds = tuple((0, None) for x in x0)
cons = ({'type': 'eq', 'fun': total_weight_constraint})
# cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1})
options = {'disp': False, 'maxiter': 1000, 'ftol': 1e-20}

solution = minimize(risk_budget_objective, x0, args=(cov), bounds=bnds, constraints=cons, method='SLSQP',
                    options=options)

# Calculate the weight
final_weights = solution.x  # weight
for i in range(len(final_weights)):
    print(f'{final_weights[i]:.1%}投资于{codes[i]}')

# Calculate the annualized return, annualized volatility, Sharpe ratio, and maximum drawdown for the risk parity portfolio
portfolio_returns = hs300_monthly * final_weights[0] + zz500_monthly * final_weights[4] + bond10_monthly * final_weights[1] + bond3_5_monthly * final_weights[4] + credit_monthly * final_weights[3]
portfolio_returns["Portfolio Return"] = portfolio_returns.sum(axis=1)
portfolio_returns["Portfolio Cumulative Return"] = (1 + portfolio_returns["Portfolio Return"]).cumprod() - 1
portfolio_returns.dropna(inplace=True)

portfolio_yearly = portfolio_returns.resample("Y").last()
portfolio_yearly["Yearly Return"] = portfolio_yearly["Portfolio Cumulative Return"].pct_change()
portfolio_yearly.fillna(0, inplace=True
                       )
portfolio_annualized_return = np.power(1 + portfolio_yearly["Yearly Return"], 12) - 1
portfolio_annualized_volatility = np.sqrt(12) * portfolio_yearly["Yearly Return"].std()
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility
portfolio_sharpe_ratio.replace([np.inf, -np.inf], np.nan, inplace=True)
portfolio_sharpe_ratio.fillna(0, inplace=True)

portfolio_max_drawdown = (portfolio_yearly["Portfolio Cumulative Return"].div(
    portfolio_yearly["Portfolio Cumulative Return"].cummax()) - 1).min()

print('年化收益率为')
print(portfolio_yearly["Yearly Return"])
print('年化波动率为')
print(portfolio_annualized_volatility)
print('夏普比率为')
print(portfolio_sharpe_ratio)
print('最大回撤为')
print(portfolio_max_drawdown)

2.2%投资于hs300
17.9%投资于bong10
30.4%投资于bond3_5
47.5%投资于credit
1.9%投资于zz500
年化收益率为
Date
2013-12-31    0.000000e+00
2014-12-31    1.988597e+28
2015-12-31    1.404798e+30
2016-12-31    4.158438e+29
2017-12-31    7.612601e+29
2018-12-31    3.579479e+29
2019-12-31    4.764594e+29
2020-12-31    1.859237e+30
2021-12-31    6.550725e+30
2022-12-31    2.517194e+30
2023-12-31    1.191725e+05
Freq: A-DEC, Name: Yearly Return, dtype: float64
年化波动率为
6.666157721393392e+30
夏普比率为
Date
2013-12-31   -3.000229e-33
2014-12-31    0.000000e+00
2015-12-31    0.000000e+00
2016-12-31    0.000000e+00
2017-12-31    0.000000e+00
2018-12-31    0.000000e+00
2019-12-31    0.000000e+00
2020-12-31    0.000000e+00
2021-12-31    0.000000e+00
2022-12-31    0.000000e+00
2023-12-31    1.231063e+30
Freq: A-DEC, Name: Yearly Return, dtype: float64
最大回撤为
0.0


C:\Users\fortu\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\fortu\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
